In [ ]:

words = open("names.txt", "r").read().splitlines()
print(words[0])

emma


In [2]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [ ]:
import torch



number of elements  164080


In [6]:
xs,ys = [],[]
for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1= stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print("number of elements ",num)
print(xs)

number of elements  228146
tensor([ 0,  5, 13,  ..., 25, 26, 24])


In [144]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
parameters = torch.randn(size=(27,27), generator = g, requires_grad=True)

In [142]:
import torch.nn.functional as F
xenc = F.one_hot(xs,num_classes=27).float()
print(xenc[1,:], xenc.shape)

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]) torch.Size([228146, 27])


In [146]:
logits = xenc @ parameters      # 228146,27 X 27 , 27 
counts = logits.exp()
print(logits[0,:],logits.shape)
print(counts[0,:], counts.shape)
# now we need to normalize the counts
prob = counts / counts.sum(1,keepdim=True)      # probability of next character
prob

tensor([ 1.5674, -0.2373, -0.0274, -1.1008,  0.2859, -0.0296, -1.5471,  0.6049,
         0.0791,  0.9046, -0.4713,  0.7868, -0.3284, -0.4330,  1.3729,  2.9334,
         1.5618, -1.6261,  0.6772, -0.8404,  0.9849, -0.1484, -1.4795,  0.4483,
        -0.0707,  2.4968,  2.4448], grad_fn=<SelectBackward0>) torch.Size([228146, 27])
tensor([ 4.7940,  0.7888,  0.9730,  0.3326,  1.3309,  0.9708,  0.2129,  1.8311,
         1.0824,  2.4710,  0.6242,  2.1964,  0.7200,  0.6486,  3.9469, 18.7908,
         4.7673,  0.1967,  1.9683,  0.4315,  2.6775,  0.8621,  0.2277,  1.5656,
         0.9317, 12.1434, 11.5281], grad_fn=<SelectBackward0>) torch.Size([228146, 27])


tensor([[0.0607, 0.0100, 0.0123,  ..., 0.0118, 0.1537, 0.1459],
        [0.0290, 0.0796, 0.0248,  ..., 0.0118, 0.0022, 0.0472],
        [0.0312, 0.0737, 0.0484,  ..., 0.1204, 0.0469, 0.0126],
        ...,
        [0.0301, 0.0080, 0.0090,  ..., 0.0125, 0.0531, 0.0111],
        [0.0634, 0.0270, 0.0101,  ..., 0.0833, 0.0175, 0.0517],
        [0.0308, 0.0113, 0.0178,  ..., 0.0190, 0.0116, 0.0562]],
       grad_fn=<DivBackward0>)

In [147]:
loss = -prob[torch.arange(num),ys].log().mean() 
loss

tensor(3.7590, grad_fn=<NegBackward0>)

In [133]:
parameters.grad=None
loss.backward()
parameters.data -= 50*parameters.grad


### Gradient descent 

In [149]:
def GD(epoch):
    for i in range(epoch):
        logits = xenc @ parameters      # 228146,27 X 27 , 27 
        counts = logits.exp()
        prob = counts / counts.sum(1,keepdim=True)  
        loss = -prob[torch.arange(num),ys].log().mean() + 0.1 * (parameters**2).mean() 
        if i % 10==0:
            print(i , loss)    
        parameters.grad=None
        loss.backward()
        parameters.data -= 50*parameters.grad



In [150]:
GD(1000)

0 tensor(3.8556, grad_fn=<AddBackward0>)
10 tensor(2.7522, grad_fn=<AddBackward0>)
20 tensor(2.6485, grad_fn=<AddBackward0>)
30 tensor(2.6163, grad_fn=<AddBackward0>)
40 tensor(2.6027, grad_fn=<AddBackward0>)
50 tensor(2.5960, grad_fn=<AddBackward0>)
60 tensor(2.5923, grad_fn=<AddBackward0>)
70 tensor(2.5902, grad_fn=<AddBackward0>)
80 tensor(2.5889, grad_fn=<AddBackward0>)
90 tensor(2.5881, grad_fn=<AddBackward0>)
100 tensor(2.5875, grad_fn=<AddBackward0>)
110 tensor(2.5872, grad_fn=<AddBackward0>)
120 tensor(2.5869, grad_fn=<AddBackward0>)
130 tensor(2.5868, grad_fn=<AddBackward0>)
140 tensor(2.5866, grad_fn=<AddBackward0>)
150 tensor(2.5866, grad_fn=<AddBackward0>)
160 tensor(2.5865, grad_fn=<AddBackward0>)
170 tensor(2.5865, grad_fn=<AddBackward0>)
180 tensor(2.5864, grad_fn=<AddBackward0>)
190 tensor(2.5864, grad_fn=<AddBackward0>)
200 tensor(2.5864, grad_fn=<AddBackward0>)
210 tensor(2.5864, grad_fn=<AddBackward0>)
220 tensor(2.5864, grad_fn=<AddBackward0>)
230 tensor(2.5864, gra

### Predict name

In [151]:
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
    ix = 0
    name = []
    while True:
        xenc = F.one_hot(torch.tensor([ix]),num_classes=27).float()

        logits = xenc @ parameters

        count = logits.exp()
        prob = count/count.sum(dim=1,keepdim=True)

        ix = torch.multinomial(prob,num_samples=1,replacement=True,generator=g).item()
        name.append(itos[ix])
        if itos[ix]==".":
            break
    print("".join(name))


cexze.
momakurailezityha.
konimittain.
llayn.
ka.
da.
staiyaubjalerigotai.
moziellavugie.
teda.
ka.


### Predicted names of this model is equal to the predicted name of statistical model
cexze.<br>
momasurailezitynn.<br>
konimittain.<br>
llayn.<br>
ka.<br>
da.<br>
staiyaubrtthrigotai.<br>
moliellavo.<br>
ke.<br>
teda.<br>